# Building Network services



# Socket programmming

If you want to use a protocol that Python doesn't support natively (or just want to use your own protocol), you can always use the lower-level `socket` module in the standard library.

But first, a (brief) review of network protocol layers

![Image](data/img/OSI.png "OSI Stack")

## Basic socket programming

In [1]:
import socket

In [2]:
# sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
sock = socket.socket()

In [3]:
sock.connect(('www.cnn.com', 80))
http_req = '''GET / HTTP/1.1
Host: www.cnn.com
User-Agent: Intermediate-Python/1.0
Accept: */*

'''
sock.sendall(http_req.encode('utf-8'))
response = sock.recv(1024)
sock.close()
print(len(response))

450


In [5]:
print(response.decode('utf8'))

HTTP/1.1 301 Moved Permanently
Server: Varnish
Retry-After: 0
Content-Length: 0
Cache-Control: public, max-age=600
Location: https://www.cnn.com/
Accept-Ranges: bytes
Date: Thu, 21 Mar 2019 18:39:26 GMT
Via: 1.1 varnish
Connection: close
Set-Cookie: countryCode=US; Domain=.cnn.com; Path=/
Set-Cookie: geoData=seattle|WA|98134|US|NA|-700|broadband; Domain=.cnn.com; Path=/
X-Served-By: cache-sea1040-SEA
X-Cache: HIT
X-Cache-Hits: 0




### Socket programming basics:

#### Client

 - `connect()`
 - `send()`, `recv()`
 - generally does _not_ `bind()` (but may)
 
#### Server

 - `bind()` to a well-known port
 - `listen()` to set up a *connection backlog*
 - `accept()` incoming connections, returning **a new socket**

In [6]:
from contextlib import closing

def echo_server(port):
    srv = socket.socket()
    srv.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
    srv.bind(('localhost', port))
    srv.listen(1)
    with closing(srv):
        print('Waiting for connections on localhost:{}'.format(port))
        peer_sock, peer_addr = srv.accept()
        print('got connection from {}'.format(peer_addr))
    with closing(peer_sock):
        buffer = peer_sock.recv(1000)
        print('Received "{}"'.format(buffer))
        peer_sock.sendall(buffer)

In [7]:
echo_server(8042)

Waiting for connections on localhost:8042
got connection from ('127.0.0.1', 53944)
Received "b'Hello there\n'"


### Better: use a handler in a thread

In [8]:
import threading

def handle_echo(sock, addr):
    while True:
        buffer = sock.recv(1000)
        print('Received {}'.format(buffer))
        if not buffer:
            break
        sock.sendall(buffer)
        
def echo_server(port):
    srv = socket.socket()
    srv.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
    srv.bind(('localhost', port))
    srv.listen(1)
    with closing(srv):
        print('Waiting for connections on localhost:{}'.format(port))
        peer_sock, peer_addr = srv.accept()
        print('got connection from {}'.format(peer_addr))
        t = threading.Thread(target=handle_echo, args=(peer_sock, peer_addr))
        t.setDaemon(True)
        t.start()

In [9]:
echo_server(8042)

Waiting for connections on localhost:8042
got connection from ('127.0.0.1', 53958)


## Using SocketServer as a socket server framework

In [11]:
import socketserver

class MyEchoHandler(socketserver.BaseRequestHandler):
    def handle(self):
        while True:
            buffer = self.request.recv(1000)
            print('Received {}'.format(buffer))
            if not buffer:
                break
            self.request.sendall(buffer)

In [12]:
server = socketserver.TCPServer(('localhost', 8043), MyEchoHandler)

In [13]:
server.serve_forever()

Received b'Hi there\n'
Received b'Starbucks\n'
Received b''
Received b'Again\n'
Received b''
Received b'Hi\n'
Received b''


KeyboardInterrupt: 

In [14]:
class MyEchoHandler(socketserver.BaseRequestHandler):
    def handle(self):
        while True:
            buffer = self.request.recv(1000)
            print('{}: Received {}'.format(self.client_address, buffer))
            if not buffer:
                break
            self.request.sendall(buffer)
            
server = socketserver.ThreadingTCPServer(('localhost', 8044), MyEchoHandler)

In [15]:
server.serve_forever()

('127.0.0.1', 53990): Received b'Hi there\n'
('127.0.0.1', 53992): Received b'Hi there\n'
('127.0.0.1', 53990): Received b''
('127.0.0.1', 53992): Received b''


KeyboardInterrupt: 

# Lab 

Open the [socket lab][socket-lab]

[socket-lab]: ./socket-lab.ipynb